In [1]:
import ee
from drought.data import ee_climate, pipeline
from drought.visualization import plot
import geemap
import matplotlib.pyplot as plt
import pandas as pd
import importlib
from drought.visualization import ee_viz

## Prerequisites

Before running the notebook, make sure to authenticate with Google Earth 
Engine. 

You can do so by running ```earthengine authenticate``` on the command line
or uncommenting the authentication cell below. See [here](https://developers.google.com/earth-engine/guides/python_install#authentication) for more details.

### Earth Engine Initialization

In [ ]:
# Uncomment if you want to authenticate within jupyter notebook.
ee.Authenticate()

In [3]:
ee.Initialize()

In [3]:
from drought.data.aggregator import make_monthly_composite
from typing import Callable
def from_8_days_to_monthly_2(ic: ee.ImageCollection, aggregator: Callable,
                           start_date: ee.Date, end_date: ee.Date):
    '''
    Takes in image collection that has 8-day composites, and aggregates it to
    monthly.

    It firsts upsamples data to daily, assuming that 8-day is a cumulative
    composite of those 8 days, then aggregates daily data for each month using
    the provided aggregator function.
    '''
    # Assert that the start date is divisible by 8 - which means there is a
    # corresponding 8-day composite for each day, starting on the first one.
    assert start_date.getRelative("day", "year").getInfo() % 8 == 0, \
        "The first day of the time range must include the date of the first 8-day composite."  # noqa: E501

    # Step 1 - Upsample data to daily by dividing the 8-day amount equally
    # amongst the days it represents. That is 8 days - unless it's the end of
    # the year, in which case it's shorter.
    n_days = end_date.difference(start_date, 'day').subtract(1)
    days = ee.List.sequence(0, n_days).map(
        lambda n: start_date.advance(n, 'day'))

    def aggregate(date):
        date = ee.Date(date)
        # For each day, find the corresponding 8-day composite.
        # However, sometimes (I've only seen it once), a composite may be
        # missing. If that's the case, we take the values from the one before
        # it. This is why we take a look at all composites in the last 16 days,
        # and pick the most recent one.
        img = ic.filterDate(date.advance(-15, 'day'), date.advance(1, 'day')) \
                .limit(1, "system:time_start", opt_ascending=False).first()

        # Find how many days this composite represents (usually 8, but less at
        # the end of the year).
        next_year = img.date().update(day=1, month=1).advance(1, 'year')
        divide_by = next_year.difference(img.date(), 'day').min(ee.Number(8))

        # Divide each image band by the number of days.
        return img.divide(divide_by) \
                  .set("system:time_start", date.millis()) \
                  .set("system:index", date.format("YYYY-MM-dd"))
                  #.copyProperties(img, ["system:time_start", "system:index"])

    return ee.ImageCollection(days.map(aggregate))

    # Step 2 - Aggregate daily data to monthly.
    return make_monthly_composite(daily_ic, aggregator, start_date, end_date)


## Understanding from scratch

In [4]:
# First, let's get a month of ET/PET
start_date = ee.Date('2010-01-01')
end_date = ee.Date('2010-02-01')
et_monthly = ee_climate.get_monthly_evapotranspiration_data(start_date, end_date)

In [5]:
ee_viz.viz_single_polygon(et_monthly.first(), polygon_index=3, bands=['ET'], layer_names=['ET-monthly'])

Map(center=[-0.19575441633043167, -56.31072622797261], controls=(WidgetControl(options=['position', 'transpare…

In [38]:
# First, let's get a month of ET/PET
start_date = ee.Date('2010-01-01')
end_date = ee.Date('2010-02-01')
from drought.data.aggregator import make_monthly_composite
def scale(img):
    '''
    TODO: Apply quality flags. For now just scale.
    '''
    return (img.select(['ET', 'PET'])
            .multiply(0.1)
            .toFloat()
            .copyProperties(img, ["system:time_start", "system:index"]))

et_8_days_ic = ee.ImageCollection('MODIS/006/MOD16A2') \
    .map(scale) \
    .filterDate(start_date, end_date)

In [39]:
et_8_days = et_8_days_ic.toList(et_8_days_ic.size())

In [40]:
# Let's visualize the first one - first 8 days
#ee_viz.viz_all_polygons(et_8_days.first(), bands=['ET'])

map = ee_viz.viz_single_polygon(ee.Image(et_8_days.get(0)), polygon_index=3, bands=['ET'], layer_names=['ET-0'])
map = ee_viz.viz_single_polygon(ee.Image(et_8_days.get(1)), polygon_index=3, bands=['ET'], map=map, layer_names=['ET-1'])
map = ee_viz.viz_single_polygon(ee.Image(et_8_days.get(2)), polygon_index=3, bands=['ET'], map=map, layer_names=['ET-2'])
map = ee_viz.viz_single_polygon(ee.Image(et_8_days.get(3)), polygon_index=3, bands=['ET'], map=map, layer_names=['ET-3'])
map = ee_viz.viz_single_polygon(et_8_days_ic.mean(), polygon_index=3, bands=['ET'], map=map, layer_names=['ET-mean'])
map

Map(center=[-0.19575441633043167, -56.31072622797261], controls=(WidgetControl(options=['position', 'transpare…

In [7]:
# Calculate daily values
et_daily = from_8_days_to_monthly_2(et_8_days, None, start_date, end_date)

In [8]:
map = ee_viz.viz_single_polygon(et_8_days.first(), polygon_index=3, bands=['ET'])
ee_viz.viz_single_polygon(et_daily.first(), polygon_index=3, bands=['ET'], map=map, layer_names=['ET-daily'])

Map(center=[-0.19575441633043167, -56.31072622797261], controls=(WidgetControl(options=['position', 'transpare…

In [9]:
et_monthly = make_monthly_composite(et_daily, lambda x: x.sum(), start_date, end_date)

In [11]:
et_monthly

In [13]:
map = ee_viz.viz_single_polygon(et_8_days.first(), polygon_index=3, bands=['ET'])
map = ee_viz.viz_single_polygon(et_daily.first(), polygon_index=3, bands=['ET'], map=map, layer_names=['ET-daily'])
ee_viz.viz_single_polygon(et_monthly.first(), polygon_index=3, bands=['ET'], map=map, layer_names=['ET-monthly'])

Map(center=[-0.19575441633043167, -56.31072622797261], controls=(WidgetControl(options=['position', 'transpare…

In [16]:
unmasked = et_8_days.first().unmask()
map = ee_viz.viz_single_polygon(et_8_days.first(), polygon_index=3, bands=['ET'])
ee_viz.viz_single_polygon(unmasked, polygon_index=3, bands=['ET'], layer_names=['unmasked'])

Map(center=[-0.19575441633043167, -56.31072622797261], controls=(WidgetControl(options=['position', 'transpare…

In [58]:
def make_monthly_composite_2(ic: ee.ImageCollection, aggregator: Callable,
                           start_date: ee.Date, end_date: ee.Date):
    '''
    Aggregates images for each month using the aggregator function.

    Common aggregators can be mean, sum, max, etc.
    '''
    n_months = end_date.difference(start_date, 'month').round().subtract(1)
    months = ee.List.sequence(0, n_months) \
                    .map(lambda n: start_date.advance(n, 'month'))

    def aggregate(monthly_date):
        start_month = ee.Date(monthly_date)
        end_month = start_month.advance(1, 'month')
        num_of_days = end_month.difference(start_month, 'days')
        return (ic.filterDate(start_month, end_month)
                .mean()#.multiply(num_of_days)
                # IMPORTANT: Add a date property to all images.
                # We depend on this elsewhere for stacking images.
                .set("date", start_month.format("YYYY-MM"))
                .set("month", start_month.get("month"))
                .set("year", start_month.get("year"))
                .set("system:time_start", start_month.millis()))

    return ee.ImageCollection(months.map(aggregate))

In [54]:
n_months = end_date.difference(start_date, 'month').round().subtract(1)
months = ee.List.sequence(0, n_months) \
                .map(lambda n: start_date.advance(n, 'month'))
bla = ee.Date('2010-03-01')
days = bla.advance(1, 'month').difference(bla, 'days')
days

In [59]:
et_monthly_3 = make_monthly_composite_2(et_daily, lambda x: x.sum(), start_date, end_date)

In [56]:
et_monthly_2 = make_monthly_composite_2(et_daily, lambda x: x.sum(), start_date, end_date)

In [20]:
map = ee_viz.viz_single_polygon(et_8_days.first(), polygon_index=3, bands=['ET'])
map = ee_viz.viz_single_polygon(et_daily.first(), polygon_index=3, bands=['ET'], map=map, layer_names=['ET-daily'])
ee_viz.viz_single_polygon(et_monthly_2.first(), polygon_index=3, bands=['ET'], map=map, layer_names=['ET-monthly'])

Map(center=[-0.19575441633043167, -56.31072622797261], controls=(WidgetControl(options=['position', 'transpare…

In [60]:
map = ee_viz.viz_single_polygon(et_8_days_ic.mean(), polygon_index=3, bands=['ET'], layer_names=['ET-mean'])
ee_viz.viz_single_polygon(et_monthly_2.first(), polygon_index=3, bands=['ET'], map=map, layer_names=['ET-monthly'])
ee_viz.viz_single_polygon(et_monthly_3.first(), polygon_index=3, bands=['ET'], map=map, layer_names=['ET-daily-mean'])


Map(center=[-0.19575441633043167, -56.31072622797261], controls=(WidgetControl(options=['position', 'transpare…

## Repro the old ET/PET way

In [ ]:
et_mo

In [242]:
start_date = ee.Date('2010-01-01')
end_date = ee.Date('2010-03-01')
from drought.data.aggregator import make_monthly_composite
def scale(img):
    '''
    TODO: Apply quality flags. For now just scale.
    '''
    return (img.select(['ET', 'PET'])
            .multiply(0.1)
            .toFloat()
            .copyProperties(img, ["system:time_start", "system:index"]))

et_8_days = ee.ImageCollection('MODIS/006/MOD16A2') \
    .map(scale) \
    .filterDate(start_date, end_date)

In [290]:
et_8_days

In [ ]:
import folium
import geemap
avg_img = stack.first()
#p_img = ee.Image(p_yearly.get(0)).clip(ee_geoms[0])
lon, lat = geo.get_gpd_polygons().centroid.x.iloc[0], geo.get_gpd_polygons().centroid.y.iloc[0]
my_map = folium.Map(location=[lat, lon], zoom_start=8, control_scale=True)
my_map.add_ee_layer(avg_img.select('precipitation'), {"min": 270, "max": 562, 'palette': ['white', 'blue']}, name='precipitation')
my_map.add_ee_layer(avg_img.select('radiation'), {"min": 11458598, "max": 12929422, 'palette': ['white', 'red']}, name='radiation')
my_map.add_ee_layer(avg_img.select('temperature'), {"min": 296, "max": 305, 'palette': ['blue', 'red']}, name='temperature')
#my_map.add_ee_layer(ee.Image(p_yearly.get(0)).clip(ee_geoms[0]), {"min": 2800, "max": 4200, 'palette': ['white', 'blue']}, '2019')
#my_map.add_ee_layer(ee.Image(p_yearly.get(1)).clip(ee_geoms[0]), {"min": 2800, "max": 4200, 'palette': ['white', 'red']}, '2020')
#my_map.add_ee_layer(ee.Image(p_yearly.get(1)).clip(ee_geoms[0]), {"min": 2800, "max": 4200, 'palette': ['white', 'green']}, '2021')
#my_map.add_child(folium.LayerControl())
folium.LayerControl(collapsed = True).add_to(my_map)
display(my_map)

In [292]:
# convert to pandas
from drought.data.ee_converter import get_region_as_df
all_polygons_pdfs = []
for i in range(len(ee_geoms)):
    ee_region_data = et_8_days.getRegion(ee_geoms[i], scale=5000).getInfo()
    pdf = ee_array_to_df(ee_region_data, ['ET', 'PET'])
    pdf["polygon_id"] = i + 1
    all_polygons_pdfs.append(pdf)

et_pd_old = pd.concat(all_polygons_pdfs)

In [263]:
def resample_from_8_days_to_monthly(df):
    # Resample to daily with forward filling. 
    daily_ffill = df.set_index('datetime').groupby(['longitude', 'latitude']).resample('D').ffill()
    # Reset index. Has to drop latitude and longitude because they became part of the index.
    # TODO: check if they have drop param in groupby.
    daily_ffill = daily_ffill.drop(columns=['latitude', 'longitude']).reset_index()

    # Convoluted logic to figure out how to divide daily values properly. Since it's 8-day data, we 
    # mostly want to divide by 8, but in some cases - like at the end of the year, it's less than 8 days.
    resampled_groups = df.set_index("datetime").resample("D").groups
    df_groups = pd.DataFrame(resampled_groups, index=["group"]).T
    s = df_groups.groupby("group").size()
    s.name = "count"
    s_counts = df_groups.join(s, on="group")["count"]

    # Join with 'count' column, which tells us how to divide ET and PET columns.
    daily = daily_ffill.join(s_counts, on='datetime')

    daily["ET"] = daily["ET"]/daily["count"]
    daily["PET"] = daily["PET"]/daily["count"]
    # At this point, we have per-day values. Woohoo! Upsampling is f****g hard. At least in Pandas.
    
    # All that's left is to aggregate for each month.
    daily['year'] = pd.DatetimeIndex(daily['datetime']).year
    daily['month'] = pd.DatetimeIndex(daily['datetime']).month
    return daily
    daily = daily.groupby(['longitude', 'latitude', 'month', 'year']).sum(numeric_only=True).reset_index()

    # Finally, drop month and year columns, and make datetime first of the month instead.
    daily['day'] = 1
    daily['datetime'] = pd.to_datetime(daily[['month', 'day', 'year']])
    #daily["datetime"] = daily["datetime"].to_numpy().astype("datetime64[M]")
    daily = daily.drop(columns=['month', 'year', 'day', 'count']).sort_values(by=['longitude', 'latitude', 'datetime'])
    return daily

In [264]:
resample_polygon_1 = resample_from_8_days_to_monthly(et_pd_old[et_pd_old.polygon_id==1])

In [265]:
resample_polygon_1.head()

,longitude,latitude,datetime,time,month,year,ET,PET,polygon_id,count
0,-74.402963,-2.358078,2010-01-01,1262304000000,1,2010,4.925,6.2,1,8
1,-74.402963,-2.358078,2010-01-02,1262304000000,1,2010,4.925,6.2,1,8
2,-74.402963,-2.358078,2010-01-03,1262304000000,1,2010,4.925,6.2,1,8
3,-74.402963,-2.358078,2010-01-04,1262304000000,1,2010,4.925,6.2,1,8
4,-74.402963,-2.358078,2010-01-05,1262304000000,1,2010,4.925,6.2,1,8


## Get Climate Data

In [5]:
# Dates of interest.
start_date = ee.Date('2019-01-01')
end_date = ee.Date('2019-03-01')

# Get regions of interest.
ee_geoms = pipeline.get_ee_polygons()

# Specify resolution.
scale = 5000

# Fetch climate data from Earth Engine as Pandas DataFrame.
climate_monthly_pdf = ee_climate.get_monthly_climate_data_as_pdf(
        start_date, end_date, ee_geoms, scale)

In [18]:
lala = climate_monthly_pdf[climate_monthly_pdf.datetime == "2019-01-01"]
lala[lala.polygon_id==1]['PET'].mean()

64.38659428168035

In [318]:
start_date = ee.Date('2010-01-17')
end_date = ee.Date('2010-03-01')
from drought.data.aggregator import make_monthly_composite
def scale(img):
    '''
    TODO: Apply quality flags. For now just scale.
    '''
    return (img.select(['ET', 'PET'])
            .multiply(0.1)
            .toFloat()
            .copyProperties(img, ["system:time_start", "system:index"]))

et_8_days = ee.ImageCollection('MODIS/006/MOD16A2') \
    .map(scale) \
    .filterDate(start_date, end_date)

#et_8_days_daily = make_monthly_composite(et_8_days, lambda x: x.min(), start_date, end_date)

In [307]:
# Quick data check
geemap.image_props(et_8_days.first())
# Quick data stats.
img = et_8_days.first()
for region in ee_geoms:
    print(geemap.image_stats(img.clip(region), scale=5000).getInfo())
#region0
#region7 = geemap.image_stats(img.clip(ee_geoms[7]), scale=5000).getInfo()

#print('Mean monthly precipitation in Region 0: ' + str(region0['mean']['ET']))
#print('Mean monthly precipitation in Region 7: ' + str(region7['mean']['PET']))

{'max': {'ET': 41.900001525878906, 'PET': 53.900001525878906}, 'mean': {'ET': 39.22952671480495, 'PET': 49.62422845134068}, 'min': {'ET': 36.5, 'PET': 45.400001525878906}, 'std': {'ET': 0.9256055005055367, 'PET': 1.483947979658496}, 'sum': {'ET': 60650.84823774456, 'PET': 76721.5870483997}}
{'max': {'ET': 44.70000076293945, 'PET': 57}, 'mean': {'ET': 40.385032043076244, 'PET': 51.80210264074135}, 'min': {'ET': 33.20000076293945, 'PET': 45.099998474121094}, 'std': {'ET': 1.728968480115851, 'PET': 1.937018161118717}, 'sum': {'ET': 61341.062729240766, 'PET': 78682.51841927318}}
{'max': {'ET': 40.900001525878906, 'PET': 50.79999923706055}, 'mean': {'ET': 34.770980694173275, 'PET': 43.45552452851431}, 'min': {'ET': 17.299999237060547, 'PET': 22.299999237060547}, 'std': {'ET': 6.079670040571625, 'PET': 7.365276477922798}, 'sum': {'ET': 22529.959208380013, 'PET': 28157.13492861731}}
{'max': {'ET': 44.20000076293945, 'PET': 61.20000076293945}, 'mean': {'ET': 34.635075676548816, 'PET': 47.99108

In [319]:
import folium
import geemap
from drought.data.pipeline import get_gpd_polygons
avg_img = et_8_days.first()
#p_img = ee.Image(p_yearly.get(0)).clip(ee_geoms[0])
lon, lat = get_gpd_polygons().centroid.x.iloc[0], get_gpd_polygons().centroid.y.iloc[0]
my_map = folium.Map(location=[lat, lon], zoom_start=8, control_scale=True)
my_map.add_ee_layer(avg_img.select('PET').clip(ee_geoms[0]), {"min": 45, "max": 54, 'palette': ['white', 'blue']}, name='PET')
#my_map.add_ee_layer(avg_img.select('radiation'), {"min": 11458598, "max": 12929422, 'palette': ['white', 'red']}, name='radiation')
#my_map.add_ee_layer(avg_img.select('temperature'), {"min": 296, "max": 305, 'palette': ['blue', 'red']}, name='temperature')
#my_map.add_ee_layer(ee.Image(p_yearly.get(0)).clip(ee_geoms[0]), {"min": 2800, "max": 4200, 'palette': ['white', 'blue']}, '2019')
#my_map.add_ee_layer(ee.Image(p_yearly.get(1)).clip(ee_geoms[0]), {"min": 2800, "max": 4200, 'palette': ['white', 'red']}, '2020')
#my_map.add_ee_layer(ee.Image(p_yearly.get(1)).clip(ee_geoms[0]), {"min": 2800, "max": 4200, 'palette': ['white', 'green']}, '2021')
#my_map.add_child(folium.LayerControl())
folium.LayerControl(collapsed = True).add_to(my_map)
display(my_map)

/var/folders/2q/26zyw9sd5cldkn81_ymfw6dh0000gn/T/ipykernel_59005/3901276999.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lon, lat = get_gpd_polygons().centroid.x.iloc[0], get_gpd_polygons().centroid.y.iloc[0]
/var/folders/2q/26zyw9sd5cldkn81_ymfw6dh0000gn/T/ipykernel_59005/3901276999.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lon, lat = get_gpd_polygons().centroid.x.iloc[0], get_gpd_polygons().centroid.y.iloc[0]


In [310]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [297]:
start_date = ee.Date('2010-01-01')
end_date = ee.Date('2010-03-01')
pet_monthly = et_8_days
#pet_monthly = from_8_days_to_monthly_2(et_8_days, None, start_date, end_date)
#pet_monthly = ee_climate.get_monthly_evapotranspiration_data(start_date, end_date)

In [298]:
pet_monthly

In [214]:
n_months = end_date.difference(start_date, 'month').round().subtract(1)
months = ee.List.sequence(0, n_months) \
                .map(lambda n: start_date.advance(n, 'month'))
n_months

In [302]:
from drought.data.ee_converter import get_region_as_df
all_polygons_pdfs = []
for i in range(len(ee_geoms)):
    ee_region_data = et_8_days.getRegion(ee_geoms[i], scale=10000).getInfo()
    pdf = ee_array_to_df(ee_region_data, ['ET', 'PET'])
    pdf["polygon_id"] = i + 1
    all_polygons_pdfs.append(pdf)

et_pd = pd.concat(all_polygons_pdfs)
et_pd.head()

,time,datetime,month,year,longitude,latitude,ET,PET,polygon_id
0,1262304000000,2010-01-01,1,2010,-74.33559,-2.380536,39.200001,49.299999,1
2216,1262304000000,2010-01-01,1,2010,-72.808454,-1.212726,38.400002,48.500000,1
952,1262304000000,2010-01-01,1,2010,-72.628791,-1.931378,37.299999,46.400002,1
488,1262304000000,2010-01-01,1,2010,-74.245758,-2.111041,39.000000,49.599998,1
3040,1262304000000,2010-01-01,1,2010,-74.33559,-0.673736,41.000000,53.000000,1


In [303]:
et_pd.datetime.unique()

array(['2010-01-01T00:00:00.000000000', '2010-01-09T00:00:00.000000000',
       '2010-01-17T00:00:00.000000000', '2010-01-25T00:00:00.000000000',
       '2010-02-02T00:00:00.000000000', '2010-02-10T00:00:00.000000000',
       '2010-02-18T00:00:00.000000000', '2010-02-26T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [304]:
#et_pd[et_pd.polygon_id==1].groupby('month').mean()
sub_new = et_pd[et_pd.polygon_id==1].sort_values(['latitude', 'longitude', 'datetime'])
sub_new.groupby(['datetime']).count()


,time,month,year,longitude,latitude,ET,PET,polygon_id
datetime,,,,,,,,
2010-01-01,400,400,400,400,400,400,400,400
2010-01-09,398,398,398,398,398,398,398,398
2010-01-17,363,363,363,363,363,363,363,363
2010-01-25,317,317,317,317,317,317,317,317
2010-02-02,38,38,38,38,38,38,38,38
2010-02-10,324,324,324,324,324,324,324,324
2010-02-18,173,173,173,173,173,173,173,173
2010-02-26,381,381,381,381,381,381,381,381


In [285]:
sub_old = resample_polygon_1.sort_values(['latitude', 'longitude', 'datetime'])
sub_old[sub_old.month==1].groupby(['datetime']).count()

,longitude,latitude,time,month,year,ET,PET,polygon_id,count
datetime,,,,,,,,,
2010-01-01,1583,1583,1583,1583,1583,1583,1583,1583,1583
2010-01-02,1582,1582,1582,1582,1582,1582,1582,1582,1582
2010-01-03,1582,1582,1582,1582,1582,1582,1582,1582,1582
2010-01-04,1582,1582,1582,1582,1582,1582,1582,1582,1582
2010-01-05,1582,1582,1582,1582,1582,1582,1582,1582,1582
2010-01-06,1582,1582,1582,1582,1582,1582,1582,1582,1582
2010-01-07,1582,1582,1582,1582,1582,1582,1582,1582,1582
2010-01-08,1582,1582,1582,1582,1582,1582,1582,1582,1582
2010-01-09,1582,1582,1582,1582,1582,1582,1582,1582,1582


In [93]:
n_months = end_date.difference(start_date, 'month').round().subtract(1)
months = ee.List.sequence(0, n_months) \
                .map(lambda n: start_date.advance(n, 'month'))
months

In [102]:
def aggregate(monthly_date):
    date = ee.Date(monthly_date)
    return (et_8_days_daily.filterDate(date, date.advance(1, 'month')).sum()
            # IMPORTANT: Add a date property to all images.
            # We depend on this elsewhere for stacking images.
            .set("date", date.format("YYYY-MM"))
            .set("month", date.get("month"))
            .set("year", date.get("year"))
            .set("system:time_start", date.millis()))

monthly = ee.ImageCollection(months.map(aggregate))


In [105]:
# Quick data check
geemap.image_props(monthly.first())
# Quick data stats.
img = monthly.first()
region0 = geemap.image_stats(img.clip(ee_geoms[0]), scale=5000).getInfo()
region7 = geemap.image_stats(img.clip(ee_geoms[7]), scale=5000).getInfo()

print('Mean monthly precipitation in Region 0: ' + str(region0['mean']['PET']))
print('Mean monthly precipitation in Region 7: ' + str(region7['mean']['PET']))

Mean monthly precipitation in Region 0: 64.27502241478855
Mean monthly precipitation in Region 7: 272.9210456702353


In [113]:
et_8_days_daily.filterDate(ee.Date('2019-02-01'), ee.Date('2019-02-01').advance(1, 'month'))

In [112]:
img = et_8_days_daily.filterDate(ee.Date('2019-02-01'), ee.Date('2019-02-01').advance(1, 'month')).sum()
# Quick data check
#geemap.image_props(monthly.first())
# Quick data stats.
#img = monthly.first()
region0 = geemap.image_stats(img.clip(ee_geoms[0]), scale=5000).getInfo()
region7 = geemap.image_stats(img.clip(ee_geoms[7]), scale=5000).getInfo()

print('Mean monthly precipitation in Region 0: ' + str(region0['mean']['PET']))
print('Mean monthly precipitation in Region 7: ' + str(region7['mean']['PET']))


Mean monthly precipitation in Region 0: 22.471114368195014
Mean monthly precipitation in Region 7: 218.7923952666224


In [66]:
epoch = ee.Date(1548892800000)
epoch

In [176]:
bla = et_8_days_daily.filterDate(ee.Date('2019-02-01'), ee.Date('2019-02-01').advance(1, 'month'))
from drought.data.ee_converter import get_region_as_df
all_polygons_pdfs = []
for i in range(len(ee_geoms)):
    ee_region_data = bla.getRegion(ee_geoms[i], scale=5000).getInfo()
    pdf = ee_array_to_df(ee_region_data, ['ET', 'PET'])
    pdf["polygon_id"] = i + 1
    all_polygons_pdfs.append(pdf)

et_pd = pd.concat(all_polygons_pdfs)

In [177]:
et_pd

,time,datetime,month,year,longitude,latitude,ET,PET,polygon_id
1,1548979200000,2019-02-01,2,2019,-73.549564,-2.402993,23.799999,27.700001,1
1081,1548979200000,2019-02-01,2,2019,-73.549564,-1.190268,24.000000,28.400000,1
1082,1548979200000,2019-02-01,2,2019,-73.504648,-1.190268,24.299999,28.400000,1
1083,1548979200000,2019-02-01,2,2019,-73.459732,-1.190268,24.500000,28.799999,1
1084,1548979200000,2019-02-01,2,2019,-73.414817,-1.190268,19.200001,24.900000,1
...,...,...,...,...,...,...,...,...,...
502,1548979200000,2019-02-01,2,2019,-41.165298,-16.776038,27.000000,61.099998,8
501,1548979200000,2019-02-01,2,2019,-41.210214,-16.776038,21.700001,59.400002,8
500,1548979200000,2019-02-01,2,2019,-41.255129,-16.776038,21.600000,59.400002,8
509,1548979200000,2019-02-01,2,2019,-40.850888,-16.776038,19.799999,58.900002,8


In [174]:
et_pd

,time,datetime,month,year,longitude,latitude,ET,PET,polygon_id
1,1548979200000,2019-02-01,2,2019,-73.549564,-2.402993,23.799999,27.700001,1
1081,1548979200000,2019-02-01,2,2019,-73.549564,-1.190268,24.000000,28.400000,1
1082,1548979200000,2019-02-01,2,2019,-73.504648,-1.190268,24.299999,28.400000,1
1083,1548979200000,2019-02-01,2,2019,-73.459732,-1.190268,24.500000,28.799999,1
1084,1548979200000,2019-02-01,2,2019,-73.414817,-1.190268,64.300001,79.500000,1
...,...,...,...,...,...,...,...,...,...
502,1548979200000,2019-02-01,2,2019,-41.165298,-16.776038,137.599998,257.000000,8
501,1548979200000,2019-02-01,2,2019,-41.210214,-16.776038,120.400002,251.300003,8
500,1548979200000,2019-02-01,2,2019,-41.255129,-16.776038,111.300001,247.900005,8
509,1548979200000,2019-02-01,2,2019,-40.850888,-16.776038,114.099998,248.300003,8


In [169]:
et_pd[et_pd.polygon_id == 1].groupby(['latitude', 'longitude']).count()

0                     time  datetime  month  year  ET  PET  polygon_id
latitude  longitude                                                   
-2.245788 -74.111011     1         1      1     1   1    1           1
          -73.212696     1         1      1     1   1    1           1
-1.347473 -74.111011     1         1      1     1   1    1           1
          -73.212696     1         1      1     1   1    1           1

In [151]:
et_pd[et_pd.polygon_id == 1].loc[28].longitude

-73.54956388728581

In [141]:
et_pd[et_pd.polygon_id == 1].groupby(['latitude', 'longitude', 'month', 'year']).mean().reset_index()

/var/folders/2q/26zyw9sd5cldkn81_ymfw6dh0000gn/T/ipykernel_59005/3680909637.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  et_pd[et_pd.polygon_id == 1].groupby(['latitude', 'longitude', 'month', 'year']).mean().reset_index()


,latitude,longitude,month,year,ET,PET,polygon_id
0,-2.402993,-73.549564,2,2019,2.968750,3.434375,1.0
1,-2.402993,-73.504648,2,2019,3.006250,3.471875,1.0
2,-2.402993,-73.055490,2,2019,3.462500,4.075000,1.0
3,-2.402993,-73.010575,2,2019,4.375000,5.287500,1.0
4,-2.402993,-72.965659,2,2019,4.562500,5.550000,1.0
...,...,...,...,...,...,...,...
1159,-0.651279,-72.741080,2,2019,2.800000,3.525000,1.0
1160,-0.651279,-72.696164,2,2019,3.153125,3.996875,1.0
1161,-0.651279,-72.651249,2,2019,2.853125,3.615625,1.0
1162,-0.651279,-72.606333,2,2019,2.768750,3.484375,1.0


In [98]:
et_pd[et_pd.polygon_id == 1]['PET'].mean()

41.23153367661131

In [131]:
def ee_array_to_df(arr, list_of_bands):
    '''Transforms client-side ee.Image.getRegion array to pandas.DataFrame.'''
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime, month and year.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')
    df['year'] = pd.DatetimeIndex(df['datetime']).year
    df['month'] = pd.DatetimeIndex(df['datetime']).month

    # Keep the columns of interest.
    df = df[['time', 'datetime', 'month', 'year', 'longitude', 'latitude',
             *list_of_bands]].sort_values(by='datetime')

    return df

In [235]:
from drought.data.aggregator import make_monthly_composite
from typing import Callable
def from_8_days_to_monthly_2(ic: ee.ImageCollection, aggregator: Callable,
                           start_date: ee.Date, end_date: ee.Date):
    '''
    Takes in image collection that has 8-day composites, and aggregates it to
    monthly.

    It firsts upsamples data to daily, assuming that 8-day is a cumulative
    composite of those 8 days, then aggregates daily data for each month using
    the provided aggregator function.
    '''
    # Assert that the start date is divisible by 8 - which means there is a
    # corresponding 8-day composite for each day, starting on the first one.
    assert start_date.getRelative("day", "year").getInfo() % 8 == 0, \
        "The first day of the time range must include the date of the first 8-day composite."  # noqa: E501

    # Step 1 - Upsample data to daily by dividing the 8-day amount equally
    # amongst the days it represents. That is 8 days - unless it's the end of
    # the year, in which case it's shorter.
    n_days = end_date.difference(start_date, 'day').subtract(1)
    days = ee.List.sequence(0, n_days).map(
        lambda n: start_date.advance(n, 'day'))

    def aggregate(date):
        date = ee.Date(date)
        # For each day, find the corresponding 8-day composite.
        # However, sometimes (I've only seen it once), a composite may be
        # missing. If that's the case, we take the values from the one before
        # it. This is why we take a look at all composites in the last 16 days,
        # and pick the most recent one.
        img = ic.filterDate(date.advance(-15, 'day'), date.advance(1, 'day')) \
                .limit(1, "system:time_start", opt_ascending=False).first()

        # Find how many days this composite represents (usually 8, but less at
        # the end of the year).
        next_year = img.date().update(day=1, month=1).advance(1, 'year')
        divide_by = next_year.difference(img.date(), 'day').min(ee.Number(8))

        # Divide each image band by the number of days.
        return img.divide(divide_by) \
                  .set("system:time_start", date.millis()) \
                  .set("system:index", date.format("YYYY-MM-dd"))
                  #.copyProperties(img, ["system:time_start", "system:index"])

    return ee.ImageCollection(days.map(aggregate))

    # Step 2 - Aggregate daily data to monthly.
    return make_monthly_composite(daily_ic, aggregator, start_date, end_date)

### Plot total monthly averages

In [ ]:
# Calculate monthly mean for all three years in aggregate.
total_monthly_mean = climate_monthly_pdf.groupby(['month', 'polygon_id']) \
                                .mean(numeric_only=True).reset_index() \
                                .drop(columns=['year'])

In [ ]:
# Palette for plotting. Colors can be accessed by indexing - plot.palette[i]
plot.palette

In [ ]:
# Plot aggregate monthly precipitation per region.
fig = plot.barplot_per_polygon(total_monthly_mean,
                              x='month', 
                              y='precipitation', 
                              ylabel="Precipitation mm / month",
                              title="Aggregate monthly precipitation per region 2019-2022",
                              color=plot.palette[5])
# Mark the 100mm precipitation line that signifies a dry month.
[subplot.axhline(y=100, color = 'r', linestyle = 'dotted', linewidth=3) 
 for subplot in fig.get_axes()]
plt.show()

In [ ]:
# Plot aggregate monthly radiation per region.
fig = plot.barplot_per_polygon(total_monthly_mean,
                              x='month', 
                              y='radiation', 
                              ylabel="Radiation",
                              title="Mean monthly radiation per region 2019-2022",
                              color=plot.palette[0],
                              sharey=True)
plt.show()

In [ ]:
# Plot aggregate monthly temperature per region.
fig = plot.barplot_per_polygon(total_monthly_mean,
                              x='month', 
                              y='temperature', 
                              ylabel="Temperature",
                              title="Mean monthly temperature per region 2019-2022",
                              color=plot.palette[7],
                              sharey=True)
plt.show()

In [ ]:
# Plot aggregate monthly fpar per region.
fig = plot.barplot_per_polygon(total_monthly_mean,
                              x='month', 
                              y='fpar', 
                              ylabel="Fpar",
                              title="Mean monthly Fpar per region 2019-2022",
                              color=plot.palette[3],
                              sharey=True)
plt.show()